## Work done by:

#### <b><i>Cham An Fam</i></b> | <b><i>Simon Idoko</i></b> | <b><i>Ruslan Damindarov</i></b>

<h2>ABB IRB 360-3/1130</h2>
Parallel Delta Robot

#### <b><i>Imports</i></b>

In [1]:
import numpy as np
import sympy as sym
from math import atan2
from IPython.display import clear_output
from numpy import sqrt, sin, cos, tan, arcsin, arccos

#### <b><i>Utilities</i></b>

In [2]:
def Rx(q):
  T = np.array([[1,         0,          0, 0],
                [0, np.cos(q), -np.sin(q), 0],
                [0, np.sin(q),  np.cos(q), 0],
                [0,         0,          0, 1]], dtype=float)
  return T

def Ry(q):
  T = np.array([[ np.cos(q), 0, np.sin(q), 0],
                [         0, 1,         0, 0],
                [-np.sin(q), 0, np.cos(q), 0],
                [         0, 0,         0, 1]], dtype=float)
  return T

def Rz(q):
  T = np.array([[np.cos(q), -np.sin(q), 0, 0],
                [np.sin(q),  np.cos(q), 0, 0],
                [        0,          0, 1, 0],
                [        0,          0, 0, 1]], dtype=float)
  return T

def Tx(x):
  T = np.array([[1, 0, 0, x],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Ty(y):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, y],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Tz(z):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, z],
                [0, 0, 0, 1]], dtype=float)
  return T

def Rx_sym(q):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, sym.cos(q), -sym.sin(q), 0],
        [0, sym.sin(q), sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Ry_sym(q):
  return sym.Matrix(
      [[sym.cos(q), 0, sym.sin(q), 0],
        [0, 1, 0, 0],
        [-sym.sin(q), 0, sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Rz_sym(q):
  return sym.Matrix(
      [[sym.cos(q), -sym.sin(q), 0, 0],
        [sym.sin(q), sym.cos(q), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Tx_sym(s):
  return sym.Matrix(
      [[1, 0, 0, s],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Ty_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, s],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Tz_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, s],
        [0, 0, 0, 1]]
  )

In [3]:
def xy_unit_vector(angle, z):
  x = np.cos(angle)
  y = np.sin(angle)

  return np.array([x, y, z])

In [4]:
def xyz_unit_vector(ang1, ang2):
  rz   = np.sin(ang2)
  r_xy = np.cos(ang2) 
  
  x    = r_xy * np.cos(ang1)
  y    = r_xy * np.sin(ang1)
  z    = rz 

  return np.array([x, y, z])

In [5]:
def angle_vector(point, ang1, ang2, distance):
  """
  Gives the result of starting from the point {x,y,z}, then going 
  a unit distance at angle {ang1} on x-y plane and an angle {ang2} 
  off the xy plane
  """
  x, y, z = point/np.linalg.norm(point)

  xy = np.linalg.norm([x, y])
  ang_z = atan2(z,xy)
  ang_xy = atan2(y,x)

  # xyz = np.linalg.norm([x, y, z])
  unit_vect = xyz_unit_vector(ang1+ang_xy, ang2+ang_z)

  return np.asarray(point) + (distance * unit_vect)

#### <b><i>Robot Parameters</i></b>

In [6]:
f  = 370   # Base radius
rf = 300   # Length of link attached to base
re = 800   # Length of link connected with moving platform
e  = 50    # Moving platform radius

#### <b><i>Forward Kinematics Equation</i></b>

In [7]:
def FK_Leg(q):
  phi1 = np.pi/6
  phi2 = phi1 + 2*np.pi/3 
  phi3 = phi1 - 2*np.pi/3

  R = f
  r = e

  a1 = phi1
  a2 = phi2
  a3 = phi3

  q1, q2, q3 = q
  
  T_base1 = Rz_sym(phi1) @ Tx_sym(r)
  T_base2 = Rz_sym(phi2) @ Tx_sym(r)
  T_base3 = Rz_sym(phi3) @ Tx_sym(r)
  
  T_tool1 = Rz_sym(a1) @ Tx_sym(r)
  T_tool2 = Rz_sym(a2) @ Tx_sym(r)
  T_tool3 = Rz_sym(a3) @ Tx_sym(r)
  
  T1 = T_base1 @ Ry_sym(q11) @ Tx_sym(r11) @ Ry_sym(q12) @ Rz_sym(q13) @ Tx_sym(r12) @ T_tool1
  T2 = T_base2 @ Ry_sym(q21) @ Tx_sym(r21) @ Ry_sym(q22) @ Rz_sym(q23) @ Tx_sym(r22) @ T_tool2
  T3 = T_base3 @ Ry_sym(q31) @ Tx_sym(r31) @ Ry_sym(q32) @ Rz_sym(q33) @ Tx_sym(r32) @ T_tool3
  # sym.diff(T, q1)
  return T1, T2, T3

In [8]:
def FK_sym1 ():
  phi1, phi2, phi3, R, r, a1, a2, a3, q1, q2, q3, r1, r2 = sym.symbols("phi_1 phi_2 phi_3 R r a_1 a_2 a_3 q_1 q_2 q3 r_1 r_2", real=True)

  T_base1 = Rz_sym(phi1) @ Tx_sym(r)
  T_base2 = Rz_sym(phi2) @ Tx_sym(r)
  T_base3 = Rz_sym(phi3) @ Tx_sym(r)
  
  T_tool1 = Rz_sym(a1) @ Tx_sym(r)
  T_tool2 = Rz_sym(a2) @ Tx_sym(r)
  T_tool3 = Rz_sym(a3) @ Tx_sym(r)
  
  T1 = T_base1 @ Ry_sym(q1) @ Tx_sym(r1) @ Ry_sym(q2) @ Rz_sym(q3) @ Tx_sym(r2) @ T_tool1
  T2 = T_base2 @ Ry_sym(q1) @ Tx_sym(r1) @ Ry_sym(q2) @ Rz_sym(q3) @ Tx_sym(r2) @ T_tool2
  T3 = T_base3 @ Ry_sym(q1) @ Tx_sym(r1) @ Ry_sym(q2) @ Rz_sym(q3) @ Tx_sym(r2) @ T_tool3
  
  T = Ry_sym(q1) @ Tx_sym(r1) @ Ry_sym(q2) @ Rz_sym(q3) @ Tx_sym(r2)
  # sym.diff(T, q1)
  return sym.trigsimp(T)

In [9]:
FK_sym1()

Matrix([
[ cos(q3)*cos(q_1 + q_2), -sin(q3)*cos(q_1 + q_2), sin(q_1 + q_2),  r_1*cos(q_1) + r_2*cos(q3)*cos(q_1 + q_2)],
[                sin(q3),                 cos(q3),              0,                                r_2*sin(q3)],
[-sin(q_1 + q_2)*cos(q3),  sin(q3)*sin(q_1 + q_2), cos(q_1 + q_2), -r_1*sin(q_1) - r_2*sin(q_1 + q_2)*cos(q3)],
[                      0,                       0,              0,                                          1]])

In [10]:
def FK(q):
  tan30 = tan(np.pi/6)
  tan60 = tan(np.pi/3)
  sin30 = sin(np.pi/6)

  t = (f-e) * tan30 / 2
  
  dtr = np.pi/180

  theta1, theta2, theta3 = q

  theta1 *= dtr
  theta2 *= dtr
  theta3 *= dtr

  y1 = -(t + rf*cos(theta1))
  z1 = -rf*sin(theta1)

  y2 = (t + rf*cos(theta2))*sin30
  x2 = y2*tan60
  z2 = -rf*sin(theta2)

  y3 = (t + rf*cos(theta3))*sin30
  x3 = -y3*tan60
  z3 = -rf*sin(theta3)

  dnm = (y2-y1)*x3-(y3-y1)*x2

  w1 = y1**2 + z1**2
  w2 = x2**2 + y2**2 + z2**2
  w3 = x3**2 + y3**2 + z3**2

  # x = (a1*z + b1)/dnm
  a1 = (z2-z1)*(y3-y1) - (z3-z1)*(y2-y1)
  b1 = -((w2-w1)*(y3-y1) - (w3-w1)*(y2-y1))/2

  # y = (a2*z + b2)/dnm
  a2 = -(z2-z1)*x3 + (z3-z1)*x2
  b2 = ((w2-w1)*x3 - (w3-w1)*x2)/2

  # a*z^2 + b*z + c = 0
  a = a1**2 + a2**2 + dnm**2
  b = 2*(a1*b1 + a2*(b2 - y1*dnm) - z1*(dnm**2))
  c = (b2-y1*dnm)**2 + b1**2 + dnm**2*(z1**2 - re**2)

  D = b**2 - 4*a*c
  if D<0:
    print("Negative discriminant")

  z0 = (-b - sqrt(D)) / (2*a)
  x0 = (a1*z0 + b1) / dnm
  y0 = (a2*z0 + b2) / dnm
  return [x0, y0, z0]

#### <b><i>Inverse Kinematics</i></b>

In [11]:
def computeJointAngles(pos):
  r1 = rf
  r2 = re
  x, y, z = pos
  q3 = arcsin(y/r2)
  
  B = (r1**2 - r2**2*cos(q3)**2 - x**2 - z**2)/(-2*r2*cos(q3))
  q2 = arccos((B - r2*cos(q3))/r1)


  a = x
  b = -z
  c = (r2**2*cos(q3)**2 - a**2 - b**2 - r1**2)/(-2*r1)
  q1 = arcsin(c/(np.sqrt(a**2 + b**2)) ) - arcsin(a/(np.sqrt(a**2 + b**2)) )
  return [q1, q2, q3]

In [12]:
phi1 = alpha1 = np.pi/6
phi2 = alpha2 = np.pi/6 + 2*np.pi/3
phi3 = alpha3 = np.pi/6 - 2*np.pi/3

T_base1 = Rz(phi1) @ Tx(f)
T_base2 = Rz(phi2) @ Tx(f)
T_base3 = Rz(phi3) @ Tx(f)

T_tool1 = Rz(alpha1) @ Tx(e)
T_tool2 = Rz(alpha2) @ Tx(e)
T_tool3 = Rz(alpha3) @ Tx(e)

T = np.array([[1,0,0, 100],
              [0,1,0, 100],
              [0,0,1, 500],
              [0,0,0,   1]])

In [13]:
def IK_new(T):
  r1 = rf
  r2 = re

  pos1 = (np.linalg.inv(T_base1) @ T @ np.linalg.inv(T_tool1)) [:3,3]
  Q1 = computeJointAngles(pos1)

  pos2 = (np.linalg.inv(T_base2) @ T @ np.linalg.inv(T_tool2)) [:3,3]
  Q2 = computeJointAngles(pos2)

  pos3 = (np.linalg.inv(T_base3) @ T @ np.linalg.inv(T_tool3)) [:3,3]
  Q3 = computeJointAngles(pos3)

  return [Q1, Q2, Q3]

#### <b><i>Jacobian Function</i></b>

In [14]:
def Jacobian(q1, q2, q3):
  theta11, theta12, theta13 = q1
  theta21, theta22, theta23 = q2
  theta31, theta32, theta33 = q3

  J1 = np.hstack([-np.cos(theta11+theta12), -np.tan(theta13), np.sin(theta11+theta12)])
  J2 = np.hstack([-np.cos(theta21+theta22), -np.tan(theta23), np.sin(theta21+theta22)])
  J3 = np.hstack([-np.cos(theta31+theta32), -np.tan(theta33), np.sin(theta31+theta32)])

  Jz = np.vstack([J1, J2, J3])

  A1 = rf * sin(2*theta11 + theta12)
  A2 = rf * sin(2*theta21 + theta22)
  A3 = rf * sin(2*theta31 + theta32)

  J_theta = np.diag([A1, A2, A3])

  J = np.linalg.inv(J_theta) @ Jz

  return J  

#### <b><i>Manipulability Function</i></b>

In [15]:
def manipulability(J):

  m = np.sqrt(np.linalg.det(np.dot(J, J.T)))

  return m

#### <b><i>Manipulability Data Generation Function</i></b>

In [16]:
z_range = np.linspace(-1100, -700, 30)
y_range = np.linspace(-800, 800, 30)
x_range = np.linspace(-800, 800, 30)

In [17]:
def manipulability_map():

  positions = []
  m_values = [] # Manipulability values 
  
  for z in z_range:
    for y in y_range:
      for x in x_range:

        pos = [x, y, z]

        T = np.array([[1,0,0,x],
                      [0,1,0,y],
                      [0,0,1,z],
                      [0,0,0,1]])
        
        q1, q2, q3 = IK_new(T)
        if not all(list(map(lambda x: np.isnan(x), np.asarray([q1, q2, q3], dtype=float).ravel()))):
          j = Jacobian(q1, q2, q3)

          m = manipulability(j)

          positions.append(pos)
          m_values.append(m)

  return [np.asarray(positions, dtype=float), np.asarray(m_values, dtype=float)]

#### <b><i>Plot of Manipulability for Robot</i></b>

In [18]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [19]:
positions, m_values = manipulability_map()
values = list(filter(lambda x: not np.isnan(x[0]) and x[0]<0.08e-6, zip(m_values,positions)))
clear_output()

In [20]:
mm_val, mm_pos = list(zip(*values))
mm_pos = np.asarray(mm_pos)

In [21]:
fig = px.scatter_3d(
    x=mm_pos[:,0],
    y=mm_pos[:,1],
    z=mm_pos[:,2],
    color=mm_val,
)
fig.update_traces(
  hovertemplate=[ f"Index: {np.round(val*1e9, 2)}n <br> position: {np.round(pos,2)}" for val,pos in zip(mm_val,mm_pos) ] 
)
fig.layout=dict(
    width=1000,
    height=600,
    scene = dict(
        camera=dict(eye={ 'x':1.25, 'y':-1.25, 'z':1.25 }),
        xaxis = dict( showticklabels=False ),
        yaxis = dict( showticklabels=False ),
        zaxis = dict( showticklabels=False ),
        aspectmode="cube",                                  
        aspectratio={ 'x':0.5, 'y':1, 'z':0.5 }),
    title="Manipulability Map",
    colorscale=dict(diverging="thermal"),
)
pio.show(fig)

#### <b><i>Robot Visualization</i></b>

In [22]:
r1 = rf
r2 = re
ratio=3

In [23]:
def Joint_Positions(q):
  phi1 = np.pi/6
  phi2 = phi1 + 2*np.pi/3 
  phi3 = phi1 - 2*np.pi/3

  R = f
  r = e

  a1 = phi1
  a2 = phi2
  a3 = phi3

  q1, q2, q3 = q
  
  base1 = (Rz(phi1) @ Tx(R))[:3,-1]
  base2 = (Rz(phi2) @ Tx(R))[:3,-1]
  base3 = (Rz(phi3) @ Tx(R))[:3,-1]

  Level0 = np.asarray([ base1, base2, base3 ]) # Joints attached to base

  Joint11 = angle_vector(base1, 0, q1, r1) 
  Joint12 = angle_vector(base2, 0, q2, r1) 
  Joint13 = angle_vector(base3, 0, q3, r1)  

  Level1 = np.asarray([ Joint11, Joint12, Joint13 ]) # 2nd Joints away from Level0 joints

  x, y, z = FK(np.rad2deg(q))

  vec1 = np.array([x,y,z]) - Joint11
  vec2 = np.array([x,y,z]) - Joint12
  vec3 = np.array([x,y,z]) - Joint13

  xy1 = np.linalg.norm(vec1[:2])
  xy2 = np.linalg.norm(vec2[:2])
  xy3 = np.linalg.norm(vec3[:2])

  ang21 = np.pi - (- q1 + atan2(vec1[2], xy1))  
  ang22 = np.pi - (- q2 + atan2(vec2[2], xy2))  
  ang23 = np.pi - (- q3 + atan2(vec3[2], xy3))  

  Joint21 = angle_vector(Joint11, 0, ang21, r2)
  Joint22 = angle_vector(Joint12, 0, ang22, r2) 
  Joint23 = angle_vector(Joint13, 0, ang23, r2)  

  Joint21 = Joint21 + (ratio*r*xy_unit_vector(a1, 0)) 
  Joint22 = Joint22 + (ratio*r*xy_unit_vector(a2, 0)) 
  Joint23 = Joint23 + (ratio*r*xy_unit_vector(a3, 0)) 

  Level2 = np.asarray([ Joint21, Joint22, Joint23 ]) # 3rd Joints away from Level0 joints 

  return [ Level0, Level1, Level2 ]

In [24]:
def cylinder3D(p0, p1, R):

  color_style = "Peach"

  p0 = np.asarray(p0)
  p1 = np.asarray(p1)

  #vector in direction of axis
  v = p1 - p0

  #find magnitude of vector
  mag = np.linalg.norm(v)

  #unit vector in direction of axis
  v = v / mag

  #make some vector not in the same direction as v
  not_v = np.array([1, 0, 0])
  if (v == not_v).all():
      not_v = np.array([0, 1, 0])

  #make vector perpendicular to v
  n1 = np.cross(v, not_v)
  #normalize n1
  n1 /= np.linalg.norm(n1)

  #make unit vector perpendicular to v and n1
  n2 = np.cross(v, n1)

  #surface ranges over t from 0 to length of axis and 0 to 2*pi
  t = np.linspace(0, mag, 2)
  theta = np.linspace(0, 2 * np.pi, 100)
  rsample = np.linspace(0, R, 2)

  #use meshgrid to make 2d arrays
  t, theta2 = np.meshgrid(t, theta)

  rsample,theta = np.meshgrid(rsample, theta)

  #generate coordinates for surface
  # "Tube"
  X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta2) * n1[i] + R * np.cos(theta2) *       n2[i] for i in [0, 1, 2]]
  # "Bottom"
  X2, Y2, Z2 = [p0[i] + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]
  # "Top"
  X3, Y3, Z3 = [p0[i] + v[i]*mag + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]

  tube = go.Surface(x=X, y=Y, z=Z, colorscale=color_style, showscale=False)
  bottom=dict(type='surface', x=X2, y=Y2, z=Z2, colorscale=color_style, showscale=False)
  top=dict(type='surface', x=X3, y=Y3, z=Z3, colorscale=color_style, showscale=False)

  return [tube, bottom, top]

In [25]:
def plot_robot(thetas):
  
  base_rad = f # Base Radius
  plat_rad = e # Platform Radius

  theta1, theta2, theta3 = thetas
  # np.deg2rad([-10,-10,-10])
  joint_positions = Joint_Positions(thetas)

  x_pos1, y_pos1, z_pos1 = joint_positions[0].T
  x_pos2, y_pos2, z_pos2 = joint_positions[1].T
  x_pos3, y_pos3, z_pos3 = joint_positions[2].T
  
  figures = []
  for (x_pos, y_pos, z_pos) in np.asarray([[x_pos1, y_pos1, z_pos1],[x_pos2, y_pos2, z_pos2],[x_pos3, y_pos3, z_pos3]]).T:
    obj = go.Scatter3d(
        x=np.round(x_pos,2),
        y=np.round(y_pos,2),
        z=z_pos-0.1,
        line=dict( color='black', width=22 ),
        # hoverinfo="text",
        # hovertext=[ f"angle: {ang}°" for ang in np.round(np.rad2deg(thetas),2) ],
        marker=dict(
            size=15,
            color=[ np.linalg.norm([x,y,z]) for x,y,z in zip(x_pos, y_pos, z_pos) ],
            colorscale='Viridis'),
        showlegend=False,
        name = f"Delta Robot"
    ); figures.append(obj)

  base = cylinder3D([0,0,-50], [0,0,50], base_rad-20)
  for obj in base: figures.append(obj)

  platform = cylinder3D([0,0,z_pos3[0]-25], [0,0,z_pos3[0]+25], 0.6*ratio*plat_rad)
  for obj in platform: figures.append(obj)

  layout = go.Layout(
      width=1000,
      height=700,
      scene = dict( 
          camera=dict( eye={ 'x':-1.25, 'y':-1.25, 'z':2 } ),
          aspectratio={ 'x':1.25, 'y':1.25, 'z':1 },
          xaxis = dict( showticklabels=False ),
          yaxis = dict( showticklabels=False ),
          zaxis = dict( showticklabels=False, range=[z_pos3[0]-50, 100] )),
      colorscale=dict(diverging="thermal"), 
      scene_xaxis_visible=False, 
      scene_yaxis_visible=False, 
      scene_zaxis_visible=False
  )
  fig = go.Figure( data=figures, layout=layout )
  fig.show()

plot_robot(np.deg2rad([-10,-10,-10]))